In [2]:
import numpy as np
import pandas as pd
import random
import math
import statistics as stat
import matplotlib.pyplot as plt
import itertools
import warnings

In [3]:
def mmn_queueing_redundancy_dos_identical(n,d,arrival_rate,service_rate,simulation_time=60,simuseed=math.nan):
    '''
    Simulating the mmn queueing system using the redundancy method, the user needs to provide
    the number of servers (n), the number of copies to send (d), arrival rate (lambda) and service rate (mu),
    in case of n >= 2, if mu is provided as a scaler, then all servers
    share the same mu, if mu is provided as a vector, then all servers may have different mu,
    the simulation time is by default 60 units, and for reproducibility purposes, the user
    can define simulation seed, by default the seed value is not set.
    
    Using the delete on start method.
    
    There are 6 inputs in this function,
    n:                the number of servers, a positive finite integer
    d:                the number of copies to send
    arrival_rate:     exponential rate of arrival
    service_rate:     exponential rate of service time
    simulation_time:  the simulation time, a positive finite number
    simuseed:         the simulation seed number
    
    
    There are 8 outputs in this function,
    event_calendar:   the event calendar for the whole simulation, containing all important
                      information needed (current time, next customer arrival time, finish time,
                      server occupancy variable, server next available time, server queue length,
                      server total, grand queue length and grand total)
    utiliz:           the utilization for each server
    ans:              the average number in the system for each server and grand average
    aql:              the average queue length for each server and grand average
    awt:              the average waiting time for each customer and grand average
    act:              the average cycle time for each customer and grand average
    no_cust_arr_comp: the table illustrating the number of arrived customers and the
                      the number of completed services for each server and grand total
    other_stat:       return other calculated statistics, for plotting purposes
    
    Code writer: Yaozu Li
    uoe S.No: 2298215
    
    '''
    
    if math.isnan(simuseed) == True:
        pass
    else:
        random.seed(simuseed)
        np.random.seed(simuseed)

    if not type(n) is int:
        raise TypeError("n needs to be integer")
    if n < 0:
        raise Exception("n needs to be positive")
    
    if d < 1 or d > n:
        raise Exception("d needs to be in the range 1 <= d <= n")
    
    lamb =  arrival_rate
    
    if isinstance(service_rate, list):
        if len(service_rate) == n:
            mu = service_rate
        elif (n > 1 and len(service_rate) == 1):
            mu = n*service_rate
        else:
            raise Exception("Unmatched cardinality for service rate and n")
    elif isinstance(service_rate, int) or isinstance(service_rate, float):
        if n == 1:
            mu = [service_rate]
        elif n > 1:
            mu = n*[service_rate]
    else:
        raise Exception("Please use int, float or list type for service rate")
    
    # Simulating the Redundancy queues
    redun_server = np.zeros(n,dtype=int)
    for i in range(1,n+1):
        exec(f'redun_queue_{i} = []')
    copyhistory = pd.DataFrame(np.zeros(d,dtype=int)).T
    
    # Record the start/queue/finish timing for each arrival
    timing_table = np.zeros(6)
    
    # Simulating the event calendar
    table_col = np.arange(n)*4+3
    avail_col = np.arange(n)*4+4
    queue_col = np.arange(n)*4+5
    total_col = np.arange(n)*4+6
    compare_col = np.insert(avail_col,0,[1,2])
    
    event_calendar = np.zeros([2,4+4*n])
    event_calendar[-1,1] = random.expovariate(lamb)
    event_calendar[-1,2] = simulation_time
    event_calendar[-1,avail_col] = math.nan
    
    no_customers = np.zeros(n+1,dtype=int)  # keep track of the number customers
    no_completed_service = np.zeros(n+1,dtype=int)   # keep track of the number of completed services
    arrival_label = 0    # keep track on each arrival's label
    
    while event_calendar[-1,0] < simulation_time:
        event_calendar = np.vstack([event_calendar,event_calendar[-1,:]])
        event_calendar[-1,0] = min(event_calendar[-2,compare_col])
        
        # If the next event is an arrival
        if event_calendar[-1,0] == event_calendar[-2,1]:
            event_calendar[-1,1] = event_calendar[-1,0]+random.expovariate(lamb)
            copy_list = np.random.choice(np.arange(n),size=d,replace=False)
            arrival_label += 1
            
            timing_table = np.vstack([timing_table,np.zeros(6)])
            timing_table[-1,0] = arrival_label
            timing_table[-1,1] = event_calendar[-1,0]
            timing_table[-1,5] = random.expovariate(1)
            
            # If all servers are occupied
            if sum(redun_server[copy_list]==0) == 0:
                event_calendar[-1,queue_col[copy_list]] += 1
                event_calendar[-1,total_col[copy_list]] += 1
                for i in copy_list:
                    eval(f'redun_queue_{i+1}').append(arrival_label)
                new_history = pd.DataFrame(copy_list).T
                new_history.index = [f"{arrival_label}"]
                copyhistory = copyhistory.append(new_history)
            
            # If only one server is available
            elif sum(redun_server[copy_list]==0) == 1:
                avail_server = copy_list[np.where(redun_server[copy_list]==0)[0][0]]
                event_calendar[-1,table_col[avail_server]] += 1
                event_calendar[-1,avail_col[avail_server]] = event_calendar[-1,0] + \
                                                        timing_table[arrival_label,5]/mu[avail_server]
                event_calendar[-1,total_col[avail_server]] += 1
                redun_server[avail_server] = arrival_label
                no_customers[avail_server] += 1
                
                timing_table[-1,2] = event_calendar[-1,0]
                timing_table[-1,4] = avail_server+1
                
            # If more than one servers are available
            else:
                avail_server = random.choice(copy_list[np.where(redun_server[copy_list]==0)[0]])
                event_calendar[-1,table_col[avail_server]] += 1
                event_calendar[-1,avail_col[avail_server]] = event_calendar[-1,0] + \
                                                        timing_table[arrival_label,5]/mu[avail_server]
                event_calendar[-1,total_col[avail_server]] += 1
                redun_server[avail_server] = arrival_label
                no_customers[avail_server] += 1
                
                timing_table[-1,2] = event_calendar[-1,0]
                timing_table[-1,4] = avail_server+1
            
        # Else if the next event is the simulation termination
        elif event_calendar[-1,0] == simulation_time:
            pass
        
        # Else if the next event is a service completion
        else:
            server_completion = np.where(event_calendar[-2,avail_col] == event_calendar[-1,0])[0][0]
            no_completed_service[server_completion] += 1
            event_calendar[-1,-1] += 1
            
            timing_table[redun_server[server_completion],3] = event_calendar[-1,0]
            
            # If there is no queue behind
            if event_calendar[-1,queue_col[server_completion]] == 0:
                event_calendar[-1,table_col[server_completion]] -= 1
                event_calendar[-1,avail_col[server_completion]] = math.nan
                event_calendar[-1,total_col[server_completion]] -= 1
                redun_server[server_completion] = 0
            
            # Else if there is a queue behind
            elif event_calendar[-1,queue_col[server_completion]] > 0:
                redun_server[server_completion] = eval(f'redun_queue_{server_completion+1}')[0]
                event_calendar[-1,avail_col[server_completion]] = event_calendar[-1,0] + \
                                                        timing_table[redun_server[server_completion],5]/mu[server_completion]
                for i in copyhistory.loc[f"{redun_server[server_completion]}"]:
                    event_calendar[-1,queue_col[i]] -= 1
                    event_calendar[-1,total_col[i]] -= 1
                    eval(f'redun_queue_{i+1}').remove(redun_server[server_completion])
                copyhistory = copyhistory.drop(f"{redun_server[server_completion]}")
                no_customers[server_completion] += 1
                
                timing_table[redun_server[server_completion],2] = event_calendar[-1,0]
                timing_table[redun_server[server_completion],4] = server_completion+1
                
    no_customers[-1] = sum(no_customers[:-1])
    no_completed_service[-1] = sum(no_completed_service[:-1])
    event_calendar = pd.DataFrame(event_calendar).tail(-1)
    event_calendar.columns = ['Time','Next Customer','Finish Time'] + \
                    list(itertools.chain.from_iterable([[f"Server{i}", \
                    f"Server{i} Available Time",f"Server{i} Queue",f"Server{i} Total"] for i in range(1,n+1)])) + \
                    ['Live_track']
    event_calendar["Grand Queue"] = event_calendar.iloc[:,queue_col].sum(axis=1)
    event_calendar["Grand Total"] = event_calendar.iloc[:,total_col].sum(axis=1)
    
    timing_table = pd.DataFrame(timing_table)
    timing_table.columns = ['Arrival_label','Start','Queue','Finish','Server','Base_exp1']
    timing_table = timing_table[timing_table['Finish'] != 0]
    
    # Concluding performance measures
    # Time dynamic
    time1 = event_calendar.iloc[:-1,0].to_numpy()
    time2 = event_calendar.iloc[1:,0].to_numpy()
    time3 = time2 - time1
    
    # Calculate the utilization for each server
    utiliz_server = event_calendar.iloc[:-1,table_col].to_numpy()
    utiliz_data = ((time3 @ utiliz_server)/simulation_time).round(3)
    utiliz = pd.DataFrame(utiliz_data).transpose()
    utiliz.columns = [f"Server{i} Utilization" for i in range(1,n+1)]
    
    # Calculating the average number in the systems for each server and grand total
    ans_server = event_calendar.iloc[:-1,np.append(total_col,event_calendar.shape[1]-1)].to_numpy()
    ans_data = ((time3 @ ans_server)/simulation_time).round(3)
    ans = pd.DataFrame(ans_data).transpose()
    ans.columns = [f"Server{i} Average number in the system" for i in range(1,n+1)]+["Grand Average number in the system"]
    
    # Calculating the average queue length
    aql_server = event_calendar.iloc[:-1,np.append(queue_col,event_calendar.shape[1]-2)].to_numpy()
    aql_data = ((time3 @ aql_server)/simulation_time).round(3)
    aql = pd.DataFrame(aql_data).transpose()
    aql.columns = [f"Server{i} Average queue length" for i in range(1,n+1)]+["Grand Average queue length"]
    
    # Average waiting time
    awt_data = np.array([])
    for i in range(1,n+1):
        awt_table = timing_table[timing_table['Server']==i]
        if awt_table.shape[0] != 0:
            awt_data = np.append(awt_data,stat.mean(awt_table['Queue']-awt_table['Start']))
        else:
            awt_data = np.append(awt_data,0)
    awt_data = np.append(awt_data,stat.mean(timing_table['Queue']-timing_table['Start']))
    awt = pd.DataFrame(awt_data.round(3)).transpose()
    awt.columns = [f"Server{i} Average waiting time" for i in range(1,n+1)]+["Grand Average waiting time"]
    
    # Average cycle time
    act_data = np.array([])
    for i in range(1,n+1):
        act_table = timing_table[timing_table['Server']==i]
        if act_table.shape[0] != 0:
            act_data = np.append(act_data,stat.mean(act_table['Finish']-act_table['Start']))
        else:
            act_data = np.append(act_data,0)
    act_data = np.append(act_data,stat.mean(timing_table['Finish']-timing_table['Start']))
    act = pd.DataFrame(act_data.round(3)).transpose()
    act.columns = [f"Server{i} Average cycle time" for i in range(1,n+1)]+["Grand Average cycle time"]
    
    # Show the number of customers arrived and completed
    no_cust_arr_comp = pd.DataFrame(np.vstack([no_customers,no_completed_service]))
    no_cust_arr_comp.columns = [f"Server{i}" for i in range(1,n+1)]+["Grand total"]
    no_cust_arr_comp.index = ["Arrived customers", "Completed services"]
    
    # Find the maximum queue length
    mql = event_calendar.iloc[:,queue_col].max()
    
    # For other statistics
    other_stat = {
        "n": n,
        "lamb": lamb,
        "mu": mu,
        "simulation_time": simulation_time,
        "table_col": table_col,
        "avail_col": avail_col,
        "queue_col": queue_col,
        "total_col": total_col,
        "compare_col": compare_col,
        "time3": time3,
        "mql": mql
    }
    
    # Return the event calendar and all performance measures table
    return event_calendar, utiliz, ans, aql, awt, act, no_cust_arr_comp, other_stat


In [4]:
def mmn_queueing_redundancy_doc_identical(n,d,arrival_rate,service_rate,simulation_time=60,simuseed=math.nan):
    '''
    Simulating the mmn queueing system using the redundancy method, the user needs to provide
    the number of servers (n), the number of copies to send (d), arrival rate (lambda) and service rate (mu),
    in case of n >= 2, if mu is provided as a scaler, then all servers
    share the same mu, if mu is provided as a vector, then all servers may have different mu,
    the simulation time is by default 60 units, and for reproducibility purposes, the user
    can define simulation seed, by default the seed value is not set.
    
    Using the delete on completion method.
    
    There are 6 inputs in this function,
    n:                the number of servers, a positive finite integer
    d:                the number of copies to send
    arrival_rate:     exponential rate of arrival
    service_rate:     exponential rate of service time
    simulation_time:  the simulation time, a positive finite number
    simuseed:         the simulation seed number
    
    
    There are 8 outputs in this function,
    event_calendar:   the event calendar for the whole simulation, containing all important
                      information needed (current time, next customer arrival time, finish time,
                      server occupancy variable, server next available time, server queue length,
                      server total, grand queue length and grand total)
    utiliz:           the utilization for each server
    ans:              the average number in the system for each server and grand average
    aql:              the average queue length for each server and grand average
    awt:              the average waiting time for each customer and grand average
    act:              the average cycle time for each customer and grand average
    no_cust_arr_comp: the table illustrating the number of arrived customers and the
                      the number of completed services for each server and grand total
    other_stat:       return other calculated statistics, for plotting purposes
    
    Code writer: Yaozu Li
    uoe S.No: 2298215
    
    '''
    
    if math.isnan(simuseed) == True:
        pass
    else:
        random.seed(simuseed)
        np.random.seed(simuseed)

    if not type(n) is int:
        raise TypeError("n needs to be integer")
    if n < 0:
        raise Exception("n needs to be positive")
    
    if d < 1 or d > n:
        raise Exception("d needs to be in the range 1 <= d <= n")
    
    lamb =  arrival_rate
    
    if isinstance(service_rate, list):
        if len(service_rate) == n:
            mu = service_rate
        elif (n > 1 and len(service_rate) == 1):
            mu = n*service_rate
        else:
            raise Exception("Unmatched cardinality for service rate and n")
    elif isinstance(service_rate, int) or isinstance(service_rate, float):
        if n == 1:
            mu = [service_rate]
        elif n > 1:
            mu = n*[service_rate]
    else:
        raise Exception("Please use int, float or list type for service rate")
    
    # Simulating the Redundancy queues
    redun_server = np.zeros(n,dtype=int)
    for i in range(1,n+1):
        exec(f'redun_queue_{i} = []')
    copyhistory = pd.DataFrame(np.zeros(d,dtype=int)).T
    
    # Record the start/queue/finish timing for each arrival
    timing_table = np.zeros(6)
    queue_record = np.zeros(n)
    
    # Simulating the event calendar
    table_col = np.arange(n)*4+3
    avail_col = np.arange(n)*4+4
    queue_col = np.arange(n)*4+5
    total_col = np.arange(n)*4+6
    compare_col = np.insert(avail_col,0,[1,2])
    
    event_calendar = np.zeros([2,4+4*n])
    event_calendar[-1,1] = random.expovariate(lamb)
    event_calendar[-1,2] = simulation_time
    event_calendar[-1,avail_col] = math.nan
    
    no_customers = np.zeros(n+1,dtype=int)  # keep track of the number customers
    no_completed_service = np.zeros(n+1,dtype=int)   # keep track of the number of completed services
    arrival_label = 0    # keep track on each arrival's label
    
    while event_calendar[-1,0] < simulation_time:
        event_calendar = np.vstack([event_calendar,event_calendar[-1,:]])
        event_calendar[-1,0] = min(event_calendar[-2,compare_col])
        
        # If the next event is an arrival
        if event_calendar[-1,0] == event_calendar[-2,1]:
            event_calendar[-1,1] = event_calendar[-1,0]+random.expovariate(lamb)
            copy_list = np.random.choice(np.arange(n),size=d,replace=False)
            arrival_label += 1
            new_history = pd.DataFrame(copy_list).T
            new_history.index = [f"{arrival_label}"]
            copyhistory = copyhistory.append(new_history)
            
            timing_table = np.vstack([timing_table,np.zeros(6)])
            timing_table[-1,0] = arrival_label
            timing_table[-1,1] = event_calendar[-1,0]
            timing_table[-1,5] = random.expovariate(1)
            queue_record = np.vstack([queue_record,np.zeros(n)])
            
            for i in copy_list:
                no_customers[i] += 1
                
                # If the server is idle
                if event_calendar[-1,table_col[i]] == 0:
                    event_calendar[-1,table_col[i]] += 1
                    event_calendar[-1,avail_col[i]] = event_calendar[-1,0]+timing_table[arrival_label,5]/mu[i]
                    event_calendar[-1,total_col[i]] += 1
                    redun_server[i] = arrival_label
                    
                    queue_record[arrival_label,i] = event_calendar[-1,0]
                    
                # If the server is occupied
                elif event_calendar[-1,table_col[i]] == 1:
                    event_calendar[-1,queue_col[i]] += 1
                    event_calendar[-1,total_col[i]] += 1
                    eval(f'redun_queue_{i+1}').append(arrival_label)
        
        # Else if the next event is the simulation termination
        elif event_calendar[-1,0] == simulation_time:
            pass
        
        # Else if the next event is a service completion
        else:
            server_completion = np.where(event_calendar[-2,avail_col] == event_calendar[-1,0])[0][0]
            no_completed_service[server_completion] += 1
            job_completion = redun_server[server_completion]
            event_calendar[-1,-1] += 1
            
            timing_table[job_completion,2] = queue_record[job_completion,server_completion]
            timing_table[job_completion,3] = event_calendar[-1,0]
            timing_table[job_completion,4] = server_completion+1
            
            # For the completion server
            # If there is no queue behind
            if event_calendar[-1,queue_col[server_completion]] == 0:
                event_calendar[-1,table_col[server_completion]] -= 1
                event_calendar[-1,avail_col[server_completion]] = math.nan
                event_calendar[-1,total_col[server_completion]] -= 1
                redun_server[server_completion] = 0
            
            # Else if there is a queue behind
            elif event_calendar[-1,queue_col[server_completion]] > 0:
                event_calendar[-1,queue_col[server_completion]] -= 1
                event_calendar[-1,total_col[server_completion]] -= 1
                redun_server[server_completion] = eval(f'redun_queue_{server_completion+1}')[0]
                eval(f'redun_queue_{server_completion+1}').remove(redun_server[server_completion])
                event_calendar[-1,avail_col[server_completion]] = event_calendar[-1,0] + \
                                                        timing_table[redun_server[server_completion],5]/mu[server_completion]
            
                queue_record[redun_server[server_completion],server_completion] = event_calendar[-1,0]
            
            # For the other servers in the copylist
            for i in copyhistory.loc[f"{job_completion}"]:
                if i != server_completion:
                    no_customers[i] -= 1
                
                    # If the completed job is undertaken
                    if redun_server[i] == job_completion:
                        
                        # If there is no queue behind
                        if event_calendar[-1,queue_col[i]] == 0:
                            event_calendar[-1,table_col[i]] -= 1
                            event_calendar[-1,avail_col[i]] = math.nan
                            event_calendar[-1,total_col[i]] -= 1
                            redun_server[i] = 0

                        # Else if there is a queue behind
                        elif event_calendar[-1,queue_col[i]] > 0:
                            event_calendar[-1,queue_col[i]] -= 1
                            event_calendar[-1,total_col[i]] -= 1
                            redun_server[i] = eval(f'redun_queue_{i+1}')[0]
                            eval(f'redun_queue_{i+1}').remove(redun_server[i])
                            event_calendar[-1,avail_col[i]] = event_calendar[-1,0] + \
                                                                    timing_table[redun_server[i],5]/mu[i]
                            
                            queue_record[redun_server[i],i] = event_calendar[-1,0]
                    
                    # Else if the completed job is not undertaken
                    elif redun_server[i] != job_completion:

                        event_calendar[-1,queue_col[i]] -= 1
                        event_calendar[-1,total_col[i]] -= 1
                        eval(f'redun_queue_{i+1}').remove(job_completion)

            copyhistory = copyhistory.drop(f"{job_completion}")


                
                
    no_customers[-1] = sum(no_customers[:-1])
    no_completed_service[-1] = sum(no_completed_service[:-1])
    event_calendar = pd.DataFrame(event_calendar).tail(-1)
    event_calendar.columns = ['Time','Next Customer','Finish Time'] + \
                    list(itertools.chain.from_iterable([[f"Server{i}", \
                    f"Server{i} Available Time",f"Server{i} Queue",f"Server{i} Total"] for i in range(1,n+1)])) + \
                    ['Live_track']
    event_calendar["Grand Queue"] = event_calendar.iloc[:,queue_col].sum(axis=1)
    event_calendar["Grand Total"] = event_calendar.iloc[:,total_col].sum(axis=1)
    
    timing_table = pd.DataFrame(timing_table)
    timing_table.columns = ['Arrival_label','Start','Queue','Finish','Server','Base_exp1']
    timing_table = timing_table[timing_table['Finish'] != 0]
    
    # Concluding performance measures
    # Time dynamic
    time1 = event_calendar.iloc[:-1,0].to_numpy()
    time2 = event_calendar.iloc[1:,0].to_numpy()
    time3 = time2 - time1
    
    # Calculate the utilization for each server
    utiliz_server = event_calendar.iloc[:-1,table_col].to_numpy()
    utiliz_data = ((time3 @ utiliz_server)/simulation_time).round(3)
    utiliz = pd.DataFrame(utiliz_data).transpose()
    utiliz.columns = [f"Server{i} Utilization" for i in range(1,n+1)]
    
    # Calculating the average number in the systems for each server and grand total
    ans_server = event_calendar.iloc[:-1,np.append(total_col,event_calendar.shape[1]-1)].to_numpy()
    ans_data = ((time3 @ ans_server)/simulation_time).round(3)
    ans = pd.DataFrame(ans_data).transpose()
    ans.columns = [f"Server{i} Average number in the system" for i in range(1,n+1)]+["Grand Average number in the system"]
    
    # Calculating the average queue length
    aql_server = event_calendar.iloc[:-1,np.append(queue_col,event_calendar.shape[1]-2)].to_numpy()
    aql_data = ((time3 @ aql_server)/simulation_time).round(3)
    aql = pd.DataFrame(aql_data).transpose()
    aql.columns = [f"Server{i} Average queue length" for i in range(1,n+1)]+["Grand Average queue length"]
    
    # Average waiting time
    awt_data = np.array([])
    for i in range(1,n+1):
        awt_table = timing_table[timing_table['Server']==i]
        if awt_table.shape[0] != 0:
            awt_data = np.append(awt_data,stat.mean(awt_table['Queue']-awt_table['Start']))
        else:
            awt_data = np.append(awt_data,0)
    awt_data = np.append(awt_data,stat.mean(timing_table['Queue']-timing_table['Start']))
    awt = pd.DataFrame(awt_data.round(3)).transpose()
    awt.columns = [f"Server{i} Average waiting time" for i in range(1,n+1)]+["Grand Average waiting time"]
    
    # Average cycle time
    act_data = np.array([])
    for i in range(1,n+1):
        act_table = timing_table[timing_table['Server']==i]
        if act_table.shape[0] != 0:
            act_data = np.append(act_data,stat.mean(act_table['Finish']-act_table['Start']))
        else:
            act_data = np.append(act_data,0)
    act_data = np.append(act_data,stat.mean(timing_table['Finish']-timing_table['Start']))
    act = pd.DataFrame(act_data.round(3)).transpose()
    act.columns = [f"Server{i} Average cycle time" for i in range(1,n+1)]+["Grand Average cycle time"]

    # Show the number of customers arrived and completed
    no_cust_arr_comp = pd.DataFrame(np.vstack([no_customers,no_completed_service]))
    no_cust_arr_comp.columns = [f"Server{i}" for i in range(1,n+1)]+["Grand total"]
    no_cust_arr_comp.index = ["Arrived customers", "Completed services"]
    
    # Find the maximum queue length
    mql = event_calendar.iloc[:,queue_col].max()
    
    # For other statistics
    other_stat = {
        "n": n,
        "lamb": lamb,
        "mu": mu,
        "simulation_time": simulation_time,
        "table_col": table_col,
        "avail_col": avail_col,
        "queue_col": queue_col,
        "total_col": total_col,
        "compare_col": compare_col,
        "time3": time3,
        "mql": mql
    }
    
    # Return the event calendar and all performance measures table
    return event_calendar, utiliz, ans, aql, awt, act, no_cust_arr_comp, other_stat


In [5]:
aaa = mmn_queueing_redundancy_dos_identical(5,3,20,[4,7,2,8,5],simulation_time=20,simuseed=8 )
bbb = mmn_queueing_redundancy_doc_identical(5,5,20,[4,7,2,8,5],simulation_time=20,simuseed=8)

In [12]:
aaa[1]

,Server1 Utilization,Server2 Utilization,Server3 Utilization,Server4 Utilization,Server5 Utilization
0,0.815,0.71,0.907,0.699,0.83


In [13]:
bbb[1]

,Server1 Utilization,Server2 Utilization,Server3 Utilization,Server4 Utilization,Server5 Utilization
0,0.99,0.99,0.99,0.99,0.99


In [14]:
aaa[2]

,Server1 Average number in the system,Server2 Average number in the system,Server3 Average number in the system,Server4 Average number in the system,Server5 Average number in the system,Grand Average number in the system
0,2.646,2.269,2.684,2.194,2.738,12.531


In [15]:
bbb[2]

,Server1 Average number in the system,Server2 Average number in the system,Server3 Average number in the system,Server4 Average number in the system,Server5 Average number in the system,Grand Average number in the system
0,113.657,113.657,113.657,113.657,113.657,568.285


In [16]:
aaa[3]

,Server1 Average queue length,Server2 Average queue length,Server3 Average queue length,Server4 Average queue length,Server5 Average queue length,Grand Average queue length
0,1.831,1.558,1.777,1.495,1.908,8.57


In [17]:
bbb[3]

,Server1 Average queue length,Server2 Average queue length,Server3 Average queue length,Server4 Average queue length,Server5 Average queue length,Grand Average queue length
0,112.667,112.667,112.667,112.667,112.667,563.337


In [18]:
aaa[4]

,Server1 Average waiting time,Server2 Average waiting time,Server3 Average waiting time,Server4 Average waiting time,Server5 Average waiting time,Grand Average waiting time
0,0.125,0.129,0.129,0.125,0.189,0.139


In [19]:
bbb[4]

,Server1 Average waiting time,Server2 Average waiting time,Server3 Average waiting time,Server4 Average waiting time,Server5 Average waiting time,Grand Average waiting time
0,0.0,0.0,0.0,5.362,0.0,5.362


In [20]:
aaa[5]

,Server1 Average cycle time,Server2 Average cycle time,Server3 Average cycle time,Server4 Average cycle time,Server5 Average cycle time,Grand Average cycle time
0,0.408,0.26,0.604,0.241,0.399,0.333


In [21]:
bbb[5]

,Server1 Average cycle time,Server2 Average cycle time,Server3 Average cycle time,Server4 Average cycle time,Server5 Average cycle time,Grand Average cycle time
0,0.0,0.0,0.0,5.494,0.0,5.494


In [22]:
aaa[6]

,Server1,Server2,Server3,Server4,Server5,Grand total
Arrived customers,58,107,36,121,78,400
Completed services,57,106,35,120,77,395


In [23]:
bbb[6]

,Server1,Server2,Server3,Server4,Server5,Grand total
Arrived customers,231,231,231,380,231,1304
Completed services,0,0,0,149,0,149


In [6]:
aaa[0]

,Time,Next Customer,Finish Time,Server1,Server1 Available Time,Server1 Queue,Server1 Total,Server2,Server2 Available Time,Server2 Queue,...,Server4 Available Time,Server4 Queue,Server4 Total,Server5,Server5 Available Time,Server5 Queue,Server5 Total,Live_track,Grand Queue,Grand Total
1,0.000000,0.012855,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
2,0.012855,0.176753,20.0,0.0,NaN,0.0,0.0,1.0,0.032148,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,1.0
3,0.032148,0.176753,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0
4,0.176753,0.178991,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,0.195148,0.0,1.0,0.0,NaN,0.0,0.0,1.0,0.0,1.0
5,0.178991,0.190739,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,0.195148,0.0,1.0,1.0,0.384362,0.0,1.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,19.856900,19.893618,20.0,1.0,20.260042,3.0,4.0,1.0,20.032669,3.0,...,19.971225,1.0,2.0,1.0,20.151943,3.0,4.0,394.0,12.0,17.0
799,19.893618,19.994396,20.0,1.0,20.260042,4.0,5.0,1.0,20.032669,4.0,...,19.971225,2.0,3.0,1.0,20.151943,3.0,4.0,394.0,15.0,20.0
800,19.971225,19.994396,20.0,1.0,20.260042,3.0,4.0,1.0,20.032669,3.0,...,20.148036,1.0,2.0,1.0,20.151943,3.0,4.0,395.0,12.0,17.0
801,19.994396,20.018022,20.0,1.0,20.260042,4.0,5.0,1.0,20.032669,4.0,...,20.148036,1.0,2.0,1.0,20.151943,3.0,4.0,395.0,15.0,20.0


In [7]:
bbb[0]

,Time,Next Customer,Finish Time,Server1,Server1 Available Time,Server1 Queue,Server1 Total,Server2,Server2 Available Time,Server2 Queue,...,Server4 Available Time,Server4 Queue,Server4 Total,Server5,Server5 Available Time,Server5 Queue,Server5 Total,Live_track,Grand Queue,Grand Total
1,0.000000,0.012855,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
2,0.012855,0.176753,20.0,1.0,0.046618,0.0,1.0,1.0,0.032148,0.0,...,0.029736,0.0,1.0,1.0,0.039866,0.0,1.0,0.0,0.0,5.0
3,0.029736,0.176753,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0
4,0.176753,0.237761,20.0,1.0,0.199011,0.0,1.0,1.0,0.189472,0.0,...,0.187882,0.0,1.0,1.0,0.194560,0.0,1.0,1.0,0.0,5.0
5,0.187882,0.237761,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,19.867123,19.891602,20.0,1.0,19.942300,231.0,232.0,1.0,19.902726,231.0,...,19.896131,231.0,232.0,1.0,19.923832,231.0,232.0,147.0,1155.0,1160.0
528,19.891602,20.000568,20.0,1.0,19.942300,232.0,233.0,1.0,19.902726,232.0,...,19.896131,232.0,233.0,1.0,19.923832,232.0,233.0,147.0,1160.0,1165.0
529,19.896131,20.000568,20.0,1.0,20.057636,231.0,232.0,1.0,19.988420,231.0,...,19.976883,231.0,232.0,1.0,20.025335,231.0,232.0,148.0,1155.0,1160.0
530,19.976883,20.000568,20.0,1.0,20.431673,230.0,231.0,1.0,20.236763,230.0,...,20.204278,230.0,231.0,1.0,20.340715,230.0,231.0,149.0,1150.0,1155.0
